In [ ]:
import aoc

In [ ]:
example_hex = 'D2FE28'

In [ ]:
hex_as_bin =  {'0': '0000', '1': '0001', '2': '0010', '3': '0011',
               '4': '0100','5': '0101','6': '0110','7': '0111',
               '8': '1000','9': '1001','A': '1010','B': '1011',
               'C': '1100','D': '1101','E': '1110','F': '1111'}

In [ ]:
def hex_to_bin(hex_string):
    return ''.join(hex_as_bin[h] for h in hex_string)


In [ ]:
example_bin = hex_to_bin(example_hex)

In [ ]:
example_bin

In [ ]:
def get_next_packet_and_tail(packet_string):
    version = int(packet_string[:3], 2)
    type_ID = int(packet_string[3:6], 2)
    #print(f'Running on {packet_string}')
    if type_ID == 4:
        locator = 6
        bits_of_literal = ''
        while packet_string[locator] == '1':
            bits_of_literal += packet_string[locator + 1 :locator + 5]
            locator += 5
        # one more set of bits for literal
        bits_of_literal += packet_string[locator + 1 :locator + 5]
        locator += 5
        literal = int(bits_of_literal, 2)
        print(f'This packet is a literal with bits {bits_of_literal}')
        packet = {'version': version, 'type_ID': type_ID, 'type': 'literal',
                  'literal': literal}
        tail = packet_string[locator:]
        return packet, tail
    else:
        length_type_ID = packet_string[6]
        print(type_ID, length_type_ID)
        if length_type_ID == '0':
            sub_packet_length = int(packet_string[7:22], 2)
            print(f'Next {sub_packet_length} bits are subpackets')
            sub_packet_bits = packet_string[22: 22 + sub_packet_length]
            print(f'those bits are {sub_packet_bits}')
            sub_packets = get_packets(sub_packet_bits)
            packet = {'version': version, 'type_ID': type_ID, 'type': 'operator',
                      'sub_packets': sub_packets}
            tail = packet_string[22 + sub_packet_length:]
            return packet, tail
        else:
            num_sub_packets = int(packet_string[7:18], 2)
            print(f'This operator has {num_sub_packets} sub_packets')
            tail = packet_string[18:]
            sub_packets = []
            for k in range(num_sub_packets):
                next_sub_packet, tail = get_next_packet_and_tail(tail)
                sub_packets.append(next_sub_packet)
                print(f'sub packet {k} is {next_sub_packet} leaving tail {tail}')
            packet = {'version': version, 'type_ID': type_ID, 'type': 'operator',
                      'sub_packets': sub_packets}
            return packet, tail
    

In [ ]:
def get_packets(packet_string):
    packets = []
    while len(packet_string) >= 11:
        new_packet, packet_string = get_next_packet_and_tail(packet_string)
        packets.append(new_packet)
    return packets
        

In [ ]:
get_packets(example_bin)

In [ ]:
get_packets('00111000000000000110111101000101001010010001001000000000')

In [ ]:
A='11010001010'
B= '010100100010010' 

In [ ]:
get_packets('11010001010010100100010010')

In [ ]:
get_next_packet_and_tail('11101110000000001101010000001100100000100011000001100000')

In [ ]:
example_1 = hex_to_bin('8A004A801A8002F478')

In [ ]:
packets1 = get_packets(example_1)

In [ ]:
def version_sum(packet):
    if packet['type'] == 'literal':
        return packet['version']
    elif packet['type'] == 'operator':
        return packet['version'] + sum([version_sum(p) for p in packet['sub_packets']])

In [ ]:
def packets_version_sum(packets):
    return sum([version_sum(p) for p in packets])

In [ ]:
packets_version_sum(packets1)

In [ ]:
example2 = hex_to_bin('620080001611562C8802118E34')
packets2 = get_packets(example2)
packets2

In [ ]:
packets_version_sum(packets2)

In [ ]:
example3 = hex_to_bin('C0015000016115A2E0802F182340')
packets3 = get_packets(example3)
packets3

In [ ]:
packets_version_sum(packets3)

In [ ]:
example4 = hex_to_bin('A0016C880162017C3686B18A3D4780')
packets4 = get_packets(example4)
packets4

In [ ]:
packets_version_sum(packets4)

In [ ]:
day16 = aoc.read_file_as_string('inputs/day16.txt')

In [ ]:
day16

In [ ]:
star1 = hex_to_bin(day16)
packets_s1 = get_packets(star1)
packets_s1

In [ ]:
packets_version_sum(packets_s1)

In [ ]:
def value_packet(packet):
    if packet['type'] == 'literal':
        return packet['literal']
    elif packet['type'] == 'operator':
        sub_packet_values = [value_packet(p) for p in packet['sub_packets']]
        if packet['type_ID'] == 0:
            return sum(sub_packet_values)
        elif packet['type_ID'] == 1:
            product = 1
            for v in sub_packet_values:
                product *= v
            return product
        elif packet['type_ID'] == 2:
            return min(sub_packet_values)
        elif packet['type_ID'] == 3:
            return max(sub_packet_values)
        elif packet['type_ID'] == 5:
            if sub_packet_values[0] > sub_packet_values[1]:
                return 1
            else: 
                return 0
        elif packet['type_ID'] == 6:
            if sub_packet_values[0] < sub_packet_values[1]:
                return 1
            else: 
                return 0
        elif packet['type_ID'] == 7:
            if sub_packet_values[0] == sub_packet_values[1]:
                return 1
            else: 
                return 0
        else:
            raise ValueError(f"Unexpected type_ID={packet['type_ID']}")

In [ ]:
value_packet(packets_s1[0])